# LAB 5 - Quantum Key Distribution

| Name | Surname | ID Number |
|------|---------|---------------------|
| Lorenzo | Calandra Buonaura | 2107761 |

**Import zone**

In [1]:
import qkd_functions as qkd

**Dataset loading**

In [2]:
dataset = 1

transmitted_states = qkd.dataset_loading(f"./Data/set{dataset}/states.txt", t="state")
received_states = qkd.dataset_loading(f"./Data/set{dataset}/statesRCV.txt", t="state")
decoy = qkd.dataset_loading(f"./Data/set{dataset}/decoy.txt", t="decoy")

assert(len(received_states) == len(transmitted_states) == len(decoy))

**Probabilities**

In [3]:
a_values = ['D', 'H', 'V']
b_values = ['D', 'H', 'V', 'A']
decoy_values = ['H', 'L', 'N']

states_trans_sep = qkd.divide_states(transmitted_states, a_values)
states_rec_sep = qkd.divide_states(received_states, b_values)
states_decoy_sep = qkd.divide_states(decoy, decoy_values)

In [4]:
prob_enc_X = len(states_trans_sep['D']) / len(transmitted_states)
prob_enc_Z = (len(states_trans_sep['H']) + len(states_trans_sep['V'])) / len(transmitted_states)

print(f'Probability of encoding in base X = {prob_enc_X}')
print(f'Probability of encoding in base Z = {prob_enc_Z}')
print(f'Total probability = {prob_enc_Z + prob_enc_X}')

Probability of encoding in base X = 0.10213526155021001
Probability of encoding in base Z = 0.89786473844979
Total probability = 1.0


In [5]:
prob_dec_X = (len(states_rec_sep['D']) + len(states_rec_sep['A'])) / len(received_states)
prob_dec_Z = (len(states_rec_sep['H']) + len(states_rec_sep['V'])) / len(received_states)

print(f'Probability of encoding in base X = {prob_dec_X}')
print(f'Probability of encoding in base Z = {prob_dec_Z}')
print(f'Total probability = {prob_dec_Z + prob_dec_X}')

Probability of encoding in base X = 0.5327614547537228
Probability of encoding in base Z = 0.4672385452462772
Total probability = 1.0


**Conditional probabilities**

In [6]:
cond_probs = qkd.total_cond_probs(transmitted_states, received_states, a_values, b_values)

In [7]:
cond_probs

{('D', 'D'): 0.5463499409795984,
 ('H', 'D'): 0.2444528766721279,
 ('V', 'D'): 0.20121610999944858,
 ('A', 'D'): 0.00798107234882506,
 ('D', 'H'): 0.3300246091661935,
 ('H', 'H'): 0.4367162889775544,
 ('V', 'H'): 0.009949877590409516,
 ('A', 'H'): 0.22330922426584263,
 ('D', 'V'): 0.21777153754047918,
 ('H', 'V'): 0.004738666412861399,
 ('V', 'V'): 0.4896997668219239,
 ('A', 'V'): 0.28779002922473557}

**QBER**

In [8]:
# QBER achieved in the state H, prob(a!=b|a=H)
a_val = a_values[1]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=H) = {qber}\n----------------------------')

# QBER achieved in the state V, prob(a!=b|a=V)
a_val = a_values[2]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=V) = {qber}\n----------------------------')

# QBER achieved in the state D, prob(a!=b|a=D)
a_val = a_values[0]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|a=D) = {qber}\n----------------------------')

# QBER achieved in the base X, prob(a!=b|A=X, B=X)
a_X = ['D']
b_X = ['A', 'D']
a_val = a_X[0]
b_val = [b for b in b_X if b != a_val]

cond_probs, qber = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|A=X, B=X) = {qber}\n----------------------------')

# QBER achieved in the base Z, prob(a!=b|A=Z, B=Z)
a_Z = ['H', 'V']
b_Z = ['H', 'V']

a_val = a_Z[0]
b_val = [b for b in b_Z if b != a_val]
cond_probs1, qber1 = qkd.getQBER(transmitted_states, received_states, a_val, b_val)

a_val = a_Z[1]
b_val = [b for b in b_Z if b != a_val]
cond_probs2, qber2 = qkd.getQBER(transmitted_states, received_states, a_val, b_val)
print(f'P(a!=b|A=Z, B=Z) = {qber1 + qber2}')

P(a!=b|a=H) = 0.5632837110224457
----------------------------
P(a!=b|a=V) = 0.5103002331780762
----------------------------
P(a!=b|a=D) = 0.45365005902040156
----------------------------
P(a!=b|A=X, B=X) = 0.00798107234882506
----------------------------
P(a!=b|A=Z, B=Z) = 0.014688544003270914


**Decoy state**

In [9]:
prob_decoy_high = len(states_decoy_sep['H']) / len(decoy)
prob_decoy_low = len(states_decoy_sep['L']) / len(decoy)

print(f'Probability of high intensity state = {prob_decoy_high}')
print(f'Probability of low intensity state = {prob_decoy_low}')
print(f'Total probability = {prob_decoy_high + prob_decoy_low}')

Probability of high intensity state = 0.8891890511645666
Probability of low intensity state = 0.11081094883543337
Total probability = 1.0


**Modified BB84**

Let's suppose that you have a perfect channel which does not introduce any errors, which means:

$P(a=b|A=B)=1$ and $P(b=D|A=Z)=P(b=A|A=Z)=P(b=H|A=X)=P(b=V|A=X)=0.5$.

An eavesdropper intercepts the photons along the channel fiber and he measures with the same probabilities of Bob in the base $E={X, Z}$.<br> 
After the measurement, he sends to Bob the state $e$ measured by him. <br>
What are the QBERs experienced by Alice and Bob in this intercept and resend attack? Is our system secure against this type of attack?